# Blue Club Functions

In [1]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import scipy.stats as stats
from endplay import *

In [2]:
def points_rs(hnd):
   long = 0
   short = dist_points(hnd, [2, 1, 0, 0])
   L1 = dist_points(hnd, [0, 0, 0, 0, 0, 1])
   L2 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 2])
   L3 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 3])
   L4 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 4])
   L5 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 0, 5])
   L6 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6])
   L7 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7])
   long = max(L1, L2, L3, L4, L5, L6, L7)
   if (shape(hnd) == [5,5,3,0] or shape(hnd) == [5,5,2,1]):
      long = long +1
   if (shape(hnd) == [6,5,2,0] or shape(hnd) == [6,5,1,1]):
      long = long +1
   if (shape(hnd) == [6,6,1,0]):
      long = long +2
   if (shape(hnd) == [7,5,1,0]):
      long = long +1
   if (shape(hnd) == [7,6,0,0]):
      long = long +2
   if (shape(hnd) == [8,5,0,0]):
      long = long +1
   totalPts = short + long + hcp(hnd)
   return totalPts

In [3]:
def rule20(h):
   TF = False
   if (hcp(h) + shape(h)[0] + shape(h)[1]) >= 20:
      TF = True
   return TF

In [4]:
def bid_qual(soot):
   if (len(soot) == 0):
      eval = 0 ;
   else:
      eval = len(soot) + ( hcp(soot) / 10 )
   return eval

In [5]:
def quals(h):
   q = make_array( bid_qual(h.spades) , bid_qual(h.hearts) , bid_qual(h.diamonds) , bid_qual(h.clubs) ) ;
   return q

In [6]:
def control(soot):
   length = len(soot)
   control = 0
   if (length == 0):
      control = 1
   if (length == 1 and hcp(soot) == 0):
      control = 2
   if (length == 1 and soot.__contains__('A')):
      control = 1
   if (length == 2 and soot.__contains__('A')):
      control = 1
   if (length == 2 and ( soot.__contains__('K') or soot.__contains__('A') ) ):
      control = 2
   return control

In [7]:
def make_bid_table(hnd):
   control_array = make_array( control(hnd.spades) , control(hnd.hearts) , control(hnd.diamonds) , control(hnd.clubs) )
   array_hcp = make_array(hcp(hnd.spades),hcp(hnd.hearts),hcp(hnd.diamonds),hcp(hnd.clubs))
   array_length = exact_shape(hnd)
   array_titles = make_array('Spades','Hearts','Diamonds','Clubs')
   q = quals(hnd)
   bid_tab = Table().with_columns('Suit', array_titles, 'Length', array_length, 'HCP', array_hcp, 'Controls', control_array, 'Bid Quality', q)
   return bid_tab

In [8]:
def balanced(hnd):
   ball = False
   if ( shape(hnd) == [4,3,3,3] ):
      ball = True
   elif ( shape(hnd) == [4,4,3,2] ):
      ball = True
   elif (shape(hnd) == [5,3,3,2] ):
      ball = True
   return ball

In [9]:
def semibalanced(hnd):
   ball = False ; shap = False ; pts = False
   if ( min( len(hnd.spades) , len(hnd.hearts) , len(hnd.diamonds) , len(hnd.clubs) ) >= 2 ):
      shap = True
   if ( bid_qual(hnd.spades) >= 2.3 ):
      pts = True
   if ( bid_qual(hnd.hearts) >= 2.3):
      pts = True
   if ( bid_qual(hnd.diamonds) >= 2.3):
      pts = True
   if ( bid_qual(hnd.clubs) >= 2.3 ):
      pts = True
   ball = shap and pts
   return ball

In [10]:
def notrump(hnd):
   NT = False
   if ( balanced(hnd) ):
      if (hcp(hnd) >= 15 and hcp(hnd) <= 23 ):
         NT = True
   if ( len(hnd.spades) == 2 and hcp(hnd.spades) < 3):
      NT = False
   elif(len(hnd.hearts) == 2 and hcp(hnd.hearts) < 3):
      NT = False
   elif(len(hnd.diamonds) == 2 and hcp(hnd.diamonds) < 3):
      NT = False
   elif(len(hnd.clubs) == 2 and hcp(hnd.clubs) < 3):
      NT = False 
   return NT

In [11]:
def strong_club(hnd):
   sc = False
   if ( points_rs(hnd) >= 17 ):
      sc = True
   return sc

In [12]:
def standard(hnd):
   std = False; std_pts = False;
   if ( points_rs(hnd) >= 12 and points_rs(hnd) <= 16 ):
      std_pts = True
   if ( std_pts and rule20(hnd) ):
      std = True
   return std

In [13]:
def twosuiter(hnd):
   ts = False; t = shape(hnd); temp = '' ; primary = '' ; secondary = '' ;
   if ( t[0] >= 5 and t[1] >= 4 ):
      ts = True
   temp = make_bid_table(hnd)
   if ( temp.sort(4, True).column(0).item(1) == 'clubs' ):
      primary = 'clubs'
      secondary = temp.sort(4, True).column(0).item(0)
   else:
      primary = temp.sort(4, True).column(0).item(0)
      secondary = temp.sort(4, True).column(0).item(1)
   t = {'ts': ts, 'p': primary , 's': secondary}
   return t

In [14]:
def weaktwosuiter(hnd):
   wts = False ; t = shape(hnd) ; pts = False; 
   primary = '' ; secondary = '' ;
   qual = np.sort(quals(hnd)) ;
   if ( qual.item(2) >= 4.3 and qual.item(3) >= 4.3 ):
      pts = True
   if ( pts and t[0] >= 4 and t[1] >= 4 ):
      wts = True
   if wts:
      temp = make_bid_table(hnd) ;
      primary = temp.sort(4, True).column(0).item(0)
      secondary = temp.sort(4, True).column(0).item(1)
   t = {'wts': wts, 'p': primary , 's': secondary}
   return t

In [15]:
dd1 = generate_deal('hcp(north) >= 11 && shape(north, any 44xx + any 8xxx)')
hand = dd1.north
print('\n The North hand has total points =' , points_rs(hand))
print(twosuiter(hand))
print(weaktwosuiter(hand))
dd1.pprint()
print(' The North hand is balanced: ', balanced(hand))
print(' The North hand is semibalanced: ' , semibalanced(hand))


 The North hand has total points = 13
{'ts': False, 'p': 'Spades', 's': 'Diamonds'}
{'wts': True, 'p': 'Spades', 's': 'Diamonds'}
              KQT4
              KT
              AJ92
              642
92                          873
8742                        QJ53
KQ5                         863
AQ83                        K97
              AJ65
              A96
              T74
              JT5
 The North hand is balanced:  True
 The North hand is semibalanced:  True
